In [1]:
import pickle 
import yaml
import pandas as pd
from PrepareData import prepare_data


import torch
from torch import nn, optim, Tensor
from torch.nn import functional as F
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import seaborn as sns
from architecture import CLIP
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32


In [2]:
from train_utils import load_model

In [3]:
val_ids = pickle.load(open('./checkpoints/FULL_COMBINED_RANDOM_SMILES/val_ids.pickle', 'rb'))

In [4]:
def make_deterministic(random_seed = 0):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)

In [5]:
make_deterministic(0)

In [6]:
config = yaml.safe_load(open('./checkpoints/FULL_COMBINED_RANDOM_SMILES/config.yaml', 'r'))
logs = pickle.load(open('./checkpoints/FULL_COMBINED_RANDOM_SMILES/logs.pickle', 'rb'))
for key in logs:
    if "best" in key:
        print(key, logs[key])

best_epoch 561
best_clip_epoch 578
best_recon_epoch 561
best_total_loss 0.09075611144304276
best_clip_loss 0.011152334472280928
best_recon_loss 0.0794648565351963


In [7]:
model = load_model(config['train']['checkpoint_dir'], type="best_clip")
model.eval()
dataloaders, max_charge, num_species = prepare_data(config)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

SMILES WILL BE RANDOMIZED


In [8]:
all_ids = []
with torch.no_grad():
    for i, data in tqdm(enumerate(dataloaders['val'])):    
        data = {k: v.to(device) for k, v in data.items()}
        mol_latents, spec_latents, smile_preds, logit_scale, ids = model(data)
        all_ids.append(ids.detach().cpu())
all_ids = torch.cat(all_ids, 0)

0it [00:00, ?it/s]/home2/kanakala.ganesh/miniconda3/envs/sbdd-env/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
50it [00:20,  2.49it/s]


In [9]:
count = 0
for id in val_ids:
    if id in all_ids:
        count += 1 
count

20000

In [10]:
molembeds = []
specembeds = []
with torch.no_grad():
    for i, data in tqdm(enumerate(dataloaders['train'])):    
        data = {k: v.to(device) for k, v in data.items()}
        mol_latents, spec_latents, smile_preds, logit_scale, ids = model(data)
        molembeds.append(mol_latents.detach().cpu())
        specembeds.append(spec_latents.detach().cpu())
    del mol_latents, spec_latents, smile_preds, logit_scale, ids

250it [01:28,  2.83it/s]


In [11]:
train_molembeds = torch.cat(molembeds, 0)
train_specembeds = torch.cat(specembeds, 0)
from train_utils import decoder_performance
from train_utils import top_scores, decoder_performance, distance_distribution

In [12]:
def top_scores(mat1, mat2):
    hits = []
    tops = [1,2,3,4,5,10]
    score = [0] * (len(tops))

    for i in tqdm(range(mat1.shape[0])):
        sims = mat2 @ mat1[i].t()
        for k in range(len(tops)):
            max_sims, ids = torch.topk(sims, tops[k])
            if (i) in ids:
                score[k] += 1

    for i in range(len(tops)):
        score[i] = score[i] / mat1.shape[0]

    return np.array(tops), np.array(score ) * 100

In [13]:
tops, scores = top_scores(train_specembeds, train_molembeds)
for k, acc in zip(tops, scores):
    # print("Full Top {} Spec".format(k), acc)
    print({"Full Top {} Spec Acc {}".format(k, acc)})

 16%|█▋        | 16313/100000 [00:57<04:37, 301.73it/s]